In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pbspark import from_protobuf
from bus_location_pb2 import BusLocation
import sys

In [2]:
jarsPackages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1"

In [3]:
spark = SparkSession.builder.master("local[*]") \
                    .appName('bus-location-processor') \
                    .config("spark.jars.packages", jarsPackages) \
                    .getOrCreate()

:: loading settings :: url = jar:file:/home/macbook/.local/share/virtualenvs/spark-processor-YU7I_jhp/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/macbook/.ivy2/cache
The jars for the packages stored in: /home/macbook/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dd84521c-4fe0-42db-b98d-01ef0f6735a9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.1 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in centr

22/11/01 08:28:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "103.146.34.72:9094") \
  .option("subscribe", "bus_location") \
  .option("includeHeaders", "true") \
  .load()

In [5]:
def message_proto(value):
    decoded = from_protobuf(value,BusLocation)
    return decoded

In [6]:
raw_df = df.selectExpr("value", "headers",'timestamp')

In [7]:
query = raw_df \
        .withColumn('parsed',message_proto('value')) \
        .withColumn('bus_id', expr('headers')[2]['value'].cast('string')) \
        .select('bus_id','parsed.*') \

query.printSchema()

root
 |-- bus_id: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- timestamp: long (nullable = true)



In [8]:
def writeToConsole(ds,OutputMode):
    return ds.writeStream \
            .outputMode(OutputMode) \
            .format('console') \
            .option('truncate',False) \
            .start() \
            .awaitTermination()

In [9]:
writeToConsole(query,'append')

22/11/01 08:28:27 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bb55e24a-4de1-402a-bb06-da434b483c8e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/01 08:28:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------
Batch: 0
-------------------------------------------
+------+---+----+---------+
|bus_id|lat|long|timestamp|
+------+---+----+---------+
+------+---+----+---------+



22/11/01 08:28:36 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
22/11/01 08:28:37 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
22/11/01 08:28:37 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


-------------------------------------------
Batch: 1
-------------------------------------------
+------+--------+---------+-------------+
|bus_id|lat     |long     |timestamp    |
+------+--------+---------+-------------+
|bus_33|-7.34672|112.72319|1667291311871|
|bus_33|-7.34667|112.72303|1667291312873|
|bus_33|-7.34656|112.72275|1667291313876|
+------+--------+---------+-------------+

22/11/01 08:28:37 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
22/11/01 08:28:37 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
22/11/01 08:28:37 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
-------------------------------------------
Batch: 2
---

-------------------------------------------
Batch: 4
-------------------------------------------
+------+--------+---------+-------------+
|bus_id|lat     |long     |timestamp    |
+------+--------+---------+-------------+
|bus_33|-7.34638|112.72155|1667291318886|
+------+--------+---------+-------------+

22/11/01 08:28:40 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


-------------------------------------------
Batch: 5
-------------------------------------------
+------+--------+---------+-------------+
|bus_id|lat     |long     |timestamp    |
+------+--------+---------+-------------+
|bus_33|-7.34639|112.72138|1667291319889|
+------+--------+---------+-------------+

22/11/01 08:28:41 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
-------------------------------------------
Batch: 6
-------------------------------------------
+------+--------+---------+-------------+
|bus_id|lat     |long     |timestamp    |
+------+--------+---------+-------------+
|bus_33|-7.34642|112.72123|1667291320891|
+------+--------+---------+-------------+

22/11/01 08:28:42 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
----------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/macbook/.local/share/virtualenvs/spark-processor-YU7I_jhp/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/macbook/.local/share/virtualenvs/spark-processor-YU7I_jhp/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


-------------------------------------------
Batch: 10
-------------------------------------------
+------+--------+---------+-------------+
|bus_id|lat     |long     |timestamp    |
+------+--------+---------+-------------+
|bus_33|-7.34676|112.72025|1667291324899|
+------+--------+---------+-------------+

22/11/01 08:28:46 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


-------------------------------------------
Batch: 11
-------------------------------------------
+------+-------+---------+-------------+
|bus_id|lat    |long     |timestamp    |
+------+-------+---------+-------------+
|bus_33|-7.3468|112.71997|1667291325902|
+------+-------+---------+-------------+

22/11/01 08:28:47 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


KeyboardInterrupt: 